In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.2 MB/s eta 0:00:00


In [2]:
 import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader
import optuna
from PIL import Image
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
from torch.utils.data import DataLoader, Dataset
import math
from torchvision.transforms import ColorJitter, Normalize
from torch.utils.data import ConcatDataset
from torch.utils.data import Subset
import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
low_res_folder = "./drive/MyDrive/Upscaling/mensch_low_res"
high_res_folder = "./drive/MyDrive/Upscaling/mensch"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_folder = low_res_folder
        self.high_res_folder = high_res_folder
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(self.low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(self.high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor
base_transform = transforms.Compose([
    transforms.ToTensor()
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)



# === Splitting into train and val sets ===

train_size = 0.8  # Proportion of data to be used for training
dataset_size = len(dataset)
split = int(train_size * dataset_size)
train_indices = list(range(split))
val_indices = list(range(split, dataset_size))

# Create train dataset as a subset of the combined dataset
train_dataset = Subset(dataset, train_indices)

# Create val dataset as a subset of the combined dataset
val_dataset = Subset(dataset, val_indices)



from tqdm import tqdm

# === Normalization ===
normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

train_dataset = [(normalize(image), normalize(target)) for image, target in tqdm(train_dataset, desc="Normalizing Train Dataset")]

val_dataset = [(normalize(image), normalize(target)) for image, target in tqdm(val_dataset, desc="Normalizing Validation Dataset")]





Normalizing Validation Dataset: 100%|██████████| 299/299 [03:11<00:00,  1.56it/s]


In [5]:
# === final data loaders ===

# Data loaders for train and val sets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Number of samples in each set
print(f"Number of training samples originally: {len(train_dataset)}")
print(f"Number of val samples: {len(val_dataset)}")


Number of training samples originally: 1194
Number of val samples: 299


### SRCNN

In [6]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=5, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

In [7]:
import optuna
num_epochs = 5
def optuna_train(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SRCNN().to(device)

    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    patience = 5
    best_loss = None
    best_epoch = None

    for epoch in tqdm(range(num_epochs), desc="Training"):
        model.train()
        train_loss = 0
        for input_data, desired_data in train_loader:
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            output_images = model(input_data)
            loss = criterion(output_images, desired_data)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for input_data, desired_data in val_loader:
                input_data = input_data.to(device)
                desired_data = desired_data.to(device)

                output_images = model(input_data)
                loss = criterion(output_images, desired_data)
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_loader)

        psnr = 10 * math.log10(1 / average_val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, PSNR: {psnr}")

        if best_loss is None or average_val_loss < best_loss:
            best_loss = average_val_loss
            best_epoch = epoch
        elif epoch - best_epoch > patience:
            print("Early stopping triggered.")
            break

    return best_loss

study = optuna.create_study(direction='minimize')
study.optimize(optuna_train, n_trials=20)

print('Best trial:')
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

df = study.trials_dataframe()
df.to_csv('./drive/MyDrive/Upscaling/study_results_SRCNN.csv', index=False)
#Mensch
"""Best trial:
  Value: 0.3070348620414734
  PSNR: 5.12812310084
  Params:
    lr: 0.000685586092102572
"""


[I 2023-06-23 11:54:48,696] A new study created in memory with name: no-name-a75b4449-0d96-4858-b5f2-019b95d2f974
<ipython-input-7-827007fac5ae>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
Training:  20%|██        | 1/5 [00:18<01:15, 18.77s/it]

Epoch [1/5], Training Loss: 0.3072, Validation Loss: 0.3197, PSNR: 4.952058341300595


Training:  40%|████      | 2/5 [00:28<00:40, 13.62s/it]

Epoch [2/5], Training Loss: 0.2799, Validation Loss: 0.3129, PSNR: 5.04595891765139


Training:  60%|██████    | 3/5 [00:38<00:24, 12.00s/it]

...

Epoch [3/5], Training Loss: 0.2744, Validation Loss: 0.3098, PSNR: 5.0885201484052125


Training:  80%|████████  | 4/5 [00:43<00:10, 10.74s/it]

Epoch [4/5], Training Loss: 0.2728, Validation Loss: 0.3092, PSNR: 5.097920267220719


Training: 100%|██████████| 5/5 [00:53<00:00, 10.75s/it]
[I 2023-06-23 12:12:53,699] Trial 19 finished with value: 0.308608615398407 and parameters: {'lr': 0.0003720203803525312}. Best is trial 15 with value: 0.3070348620414734.


Epoch [5/5], Training Loss: 0.2720, Validation Loss: 0.3086, PSNR: 5.1059195394442
Best trial:
  Value: 0.3070348620414734
  Params: 
    lr: 0.000685586092102572


### FSRCNN

In [ ]:
class FSRCNN(nn.Module):
    def __init__(self, d=56, s=12, m=4):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=5, padding=4, output_padding=4)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [ ]:
import optuna
num_epochs = 30

def optuna_train(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = FSRCNN().to(device)

    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    d = trial.suggest_int('d', 32, 128)
    s = trial.suggest_int('s', 5, 20)
    m = trial.suggest_int('m', 2, 8)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    patience = 5
    best_loss = None
    best_epoch = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for input_data, desired_data in train_loader:
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            output_images = model(input_data)
            loss = criterion(output_images, desired_data)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for input_data, desired_data in val_loader:
                input_data = input_data.to(device)
                desired_data = desired_data.to(device)

                output_images = model(input_data)
                loss = criterion(output_images, desired_data)
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_loader)

        psnr = 10 * math.log10(1 / average_val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, PSNR: {psnr}")

        if best_loss is None or average_val_loss < best_loss:
            best_loss = average_val_loss
            best_epoch = epoch
        elif epoch - best_epoch > patience:
            print("Early stopping triggered.")
            break

    return best_loss

study = optuna.create_study(direction='minimize')
study.optimize(optuna_train, n_trials=30)

print('Best trial:')
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

df = study.trials_dataframe()
df.to_csv('./drive/MyDrive/Upscaling/study_results_FSRCNN.csv', index=False)

#Mensch best Params
"""Best trial:
  Loss: 0.012162919548389158
  PSNR: 19.1496216589
  Params:
    lr: 0.004974424065215662
    d: 91
    s: 18
    m: 2
"""

[I 2023-06-23 06:55:54,202] A new study created in memory with name: no-name-c364b149-2e92-4930-8cf4-0cc456f24a8f
<ipython-input-9-86ddac4b39b7>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)


Epoch [1/30], Training Loss: 0.3052, Validation Loss: 0.1282, PSNR: 8.922001808515981
Epoch [2/30], Training Loss: 0.0945, Validation Loss: 0.0813, PSNR: 10.89726367502601
Epoch [3/30], Training Loss: 0.0788, Validation Loss: 0.0715, PSNR: 11.455302712281672
Epoch [4/30], Training Loss: 0.0675, Validation Loss: 0.0580, PSNR: 12.365639942400154
Epoch [5/30], Training Loss: 0.0576, Validation Loss: 0.0512, PSNR: 12.906673187042088
Epoch [6/30], Training Loss: 0.0530, Validation Loss: 0.0479, PSNR: 13.193325264386786
Epoch [7/30], Training Loss: 0.0503, Validation Loss: 0.0455, PSNR: 13.42392802460165
Epoch [8/30], Training Loss: 0.0480, Validation Loss: 0.0429, PSNR: 13.671170099789778
Epoch [9/30], Training Loss: 0.0457, Validation Loss: 0.0405, PSNR: 13.930625006054527
Epoch [10/30], Training Loss: 0.0435, Validation Loss: 0.0383, PSNR: 14.170710487089595
Epoch [11/30], Training Loss: 0.0415, Validation Loss: 0.0363, PSNR: 14.406580915893771
Epoch [12/30], Training Loss: 0.0368, Valida

[I 2023-06-23 06:56:50,246] Trial 0 finished with value: 0.01641871848780858 and parameters: {'lr': 0.0001988710796740462, 'd': 48, 's': 6, 'm': 8}. Best is trial 0 with value: 0.01641871848780858.


Epoch [30/30], Training Loss: 0.0170, Validation Loss: 0.0164, PSNR: 17.846607434037093
Epoch [1/30], Training Loss: 0.1755, Validation Loss: 0.0798, PSNR: 10.98192129910817
Epoch [2/30], Training Loss: 0.0728, Validation Loss: 0.0572, PSNR: 12.428740195933468
Epoch [3/30], Training Loss: 0.0539, Validation Loss: 0.0454, PSNR: 13.432183574717405
Epoch [4/30], Training Loss: 0.0461, Validation Loss: 0.0387, PSNR: 14.12367202459335
Epoch [5/30], Training Loss: 0.0363, Validation Loss: 0.0279, PSNR: 15.539082402067281
Epoch [6/30], Training Loss: 0.0266, Validation Loss: 0.0236, PSNR: 16.264414338797796
Epoch [7/30], Training Loss: 0.0236, Validation Loss: 0.0216, PSNR: 16.650062082381886
Epoch [8/30], Training Loss: 0.0220, Validation Loss: 0.0205, PSNR: 16.892479723863424
Epoch [9/30], Training Loss: 0.0210, Validation Loss: 0.0195, PSNR: 17.092767164486517
Epoch [10/30], Training Loss: 0.0202, Validation Loss: 0.0189, PSNR: 17.232238527218325
Epoch [11/30], Training Loss: 0.0196, Valid

[I 2023-06-23 06:57:35,130] Trial 1 finished with value: 0.014095168315658444 and parameters: {'lr': 0.0005787319068390336, 'd': 32, 's': 16, 'm': 5}. Best is trial 1 with value: 0.014095168315658444.


...

Epoch [26/30], Training Loss: 0.1415, Validation Loss: 0.1275, PSNR: 8.946433844785254
Early stopping triggered.
Epoch [1/30], Training Loss: 0.1106, Validation Loss: 0.0485, PSNR: 13.145318825939917
Epoch [2/30], Training Loss: 0.0395, Validation Loss: 0.0256, PSNR: 15.92445859560925
Epoch [3/30], Training Loss: 0.0238, Validation Loss: 0.0206, PSNR: 16.863849416549527
Epoch [4/30], Training Loss: 0.0210, Validation Loss: 0.0187, PSNR: 17.28541465121638
Epoch [5/30], Training Loss: 0.0198, Validation Loss: 0.0176, PSNR: 17.539897983204785
Epoch [6/30], Training Loss: 0.0187, Validation Loss: 0.0177, PSNR: 17.514107353622453
Epoch [7/30], Training Loss: 0.0179, Validation Loss: 0.0163, PSNR: 17.884367909037444
Epoch [8/30], Training Loss: 0.0169, Validation Loss: 0.0156, PSNR: 18.063965545668378
Epoch [9/30], Training Loss: 0.0159, Validation Loss: 0.0228, PSNR: 16.413244970904113
Epoch [10/30], Training Loss: 0.0171, Validation Loss: 0.0147, PSNR: 18.33434401942682
Epoch [11/30], Trai

[I 2023-06-23 07:16:22,798] Trial 28 finished with value: 0.012480337908001323 and parameters: {'lr': 0.004310854688616244, 'd': 57, 's': 8, 'm': 3}. Best is trial 22 with value: 0.012162919548389158.


Epoch [30/30], Training Loss: 0.0135, Validation Loss: 0.0125, PSNR: 19.037736558720496
Epoch [1/30], Training Loss: 0.1071, Validation Loss: 0.0499, PSNR: 13.022672018966661
Epoch [2/30], Training Loss: 0.0448, Validation Loss: 0.0352, PSNR: 14.539727804336263
Epoch [3/30], Training Loss: 0.0279, Validation Loss: 0.0218, PSNR: 16.611944383758434
Epoch [4/30], Training Loss: 0.0216, Validation Loss: 0.0195, PSNR: 17.102764763922668
Epoch [5/30], Training Loss: 0.0202, Validation Loss: 0.0182, PSNR: 17.409373163920066
Epoch [6/30], Training Loss: 0.0186, Validation Loss: 0.0171, PSNR: 17.67669371191632
Epoch [7/30], Training Loss: 0.0176, Validation Loss: 0.0162, PSNR: 17.915439737215607
Epoch [8/30], Training Loss: 0.0162, Validation Loss: 0.0153, PSNR: 18.158443061437424
Epoch [9/30], Training Loss: 0.0154, Validation Loss: 0.0155, PSNR: 18.099587648033943
Epoch [10/30], Training Loss: 0.0192, Validation Loss: 0.0152, PSNR: 18.16799827467343
Epoch [11/30], Training Loss: 0.0149, Valid

[I 2023-06-23 07:17:07,740] Trial 29 finished with value: 0.012755351729298892 and parameters: {'lr': 0.0024044893386930164, 'd': 40, 's': 17, 'm': 2}. Best is trial 22 with value: 0.012162919548389158.


Epoch [30/30], Training Loss: 0.0131, Validation Loss: 0.0128, PSNR: 18.92645617614937
Best trial:
  Value: 0.012162919548389158
  Params: 
    lr: 0.004974424065215662
    d: 91
    s: 18
    m: 2
